*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [1]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

/bin/bash: line 1: pip: command not found
/bin/bash: line 1: pip: command not found
2.5.0


In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2021-12-04 07:45:41--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K   623KB/s    in 0.6s    

2021-12-04 07:45:43 (623 KB/s) - ‘train-data.tsv’ saved [358233/358233]

--2021-12-04 07:45:43--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[=============

In [3]:
df_train = pd.read_csv(train_file_path,sep='\t',header=None)
df_test = pd.read_csv(test_file_path,sep='\t',header=None)
df_train[0] = df_train[0].replace("ham", 0)
df_train[0] = df_train[0].replace("spam", 1)
df_test[0] = df_test[0].replace("ham", 0)
df_test[0] = df_test[0].replace("spam", 1)
df_test[0]=df_test[0].astype('int64')
df_train[0]=df_train[0].astype('int64')
df_train.head()

,0,1
0,0,ahhhh...just woken up!had a bad dream about u ...
1,0,you can never do nothing
2,0,"now u sound like manky scouse boy steve,like! ..."
3,0,mum say we wan to go then go... then she can s...
4,0,never y lei... i v lazy... got wat? dat day ü ...


In [4]:
train_dataset = tf.data.Dataset.from_tensor_slices((df_train[1], df_train[0]))
test_dataset = tf.data.Dataset.from_tensor_slices((df_test[1], df_test[0]))
train_dataset.element_spec

(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [5]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [6]:
for example, label in train_dataset.take(1):
  print('texts: ', example.numpy()[:3])
  print()
  print('labels: ', label.numpy()[:3])


texts:  [b'k give me a sec, breaking a  &lt;#&gt;  at cstore'
 b'well done! your 4* costa del sol holiday or \xc2\xa35000 await collection. call 09050090044 now toclaim. sae, tcs, pobox334, stockport, sk38xh, cost\xc2\xa31.50/pm, max10mins'
 b'nice talking to you! please dont forget my pix :) i want to see all of you...']

labels:  [0 1 0]


In [7]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))


In [8]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]


array(['', '[UNK]', 'to', 'i', 'you', 'a', 'the', 'u', 'and', 'in', 'is',
       'me', 'my', 'for', 'your', 'of', 'it', 'call', 'have', 'on'],
      dtype='<U15')

In [9]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

array([[189, 142,  11,   5,   1,   1,   5,  48,  28,   1,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [105, 259,  14,  45,   1, 897,   1, 266,  29, 540, 569, 452,  17,
          1,  22,   1, 549, 378,   1,   1,   1,   1,   1,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [200, 658,   2,   4, 110,  47, 639,  12,   1,   3,  73,   2,  85,
         53,  15,   4,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0]])

In [10]:
for n in range(3):
  print("Original: ", example[n].numpy())
  print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
  print()


Original:  b'k give me a sec, breaking a  &lt;#&gt;  at cstore'
Round-trip:  k give me a [UNK] [UNK] a ltgt at [UNK]                       

Original:  b'well done! your 4* costa del sol holiday or \xc2\xa35000 await collection. call 09050090044 now toclaim. sae, tcs, pobox334, stockport, sk38xh, cost\xc2\xa31.50/pm, max10mins'
Round-trip:  well done your 4 [UNK] del [UNK] holiday or £5000 await collection call [UNK] now [UNK] sae tcs [UNK] [UNK] [UNK] [UNK] [UNK]          

Original:  b'nice talking to you! please dont forget my pix :) i want to see all of you...'
Round-trip:  nice talking to you please dont forget my [UNK] i want to see all of you                 



In [11]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(len(encoder.get_vocabulary()), 64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1,activation='sigmoid')
])


In [12]:
print([layer.supports_masking for layer in model.layers])


[False, True, True, True, True, True, True]


In [13]:
sample_text = ("how are you doing today?")
predictions = model.predict(np.array([sample_text]))
print(predictions[0])


[0.49877986]


In [14]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])


In [15]:
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)


Epoch 1/10


/home/spooderman/.local/lib/python3.9/site-packages/tensorflow/python/keras/backend.py:5016: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(


66/66 [==============================] - 28s 227ms/step - loss: 0.6572 - accuracy: 0.8430 - val_loss: 0.5882 - val_accuracy: 0.8657
Epoch 2/10
66/66 [==============================] - 9s 134ms/step - loss: 0.5120 - accuracy: 0.8660
Epoch 3/10
66/66 [==============================] - 9s 130ms/step - loss: 0.3810 - accuracy: 0.8674
Epoch 4/10
66/66 [==============================] - 9s 132ms/step - loss: 0.1998 - accuracy: 0.9550
Epoch 5/10
66/66 [==============================] - 8s 124ms/step - loss: 0.1121 - accuracy: 0.9794
Epoch 6/10
66/66 [==============================] - 8s 125ms/step - loss: 0.0815 - accuracy: 0.9832
Epoch 7/10
66/66 [==============================] - 8s 127ms/step - loss: 0.0656 - accuracy: 0.9876
Epoch 8/10
66/66 [==============================] - 9s 127ms/step - loss: 0.0554 - accuracy: 0.9880
Epoch 9/10
66/66 [==============================] - 8s 125ms/step - loss: 0.0466 - accuracy: 0.9902
Epoch 10/10
66/66 [==============================] - 8s 127ms/step -

In [16]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)


22/22 [==============================] - 1s 38ms/step - loss: 0.0574 - accuracy: 0.9849
Test Loss: 0.05738411098718643
Test Accuracy: 0.9849137663841248


In [17]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  prediction = []
  predictions = model.predict(np.array([pred_text]))[0]
  pre = predictions[0]
  prediction.append(pre)
  if predictions > 0.5:
    prediction.append("spam")
  else:
    prediction.append("ham")

  return (prediction)

pred_text = "how are you doing today?"


prediction = predict_message(pred_text)
print(prediction)

[0.0027385354, 'ham']


In [18]:
a = []
#a = [5,4]
a.append([5,4])
print(a)
#a = [[4,6]]
a.append([6,7])
print(a)

[[5, 4]]
[[5, 4], [6, 7]]


In [19]:
test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]

for msg, ans in zip(test_messages, test_answers):
  print(msg,ans)

how are you doing today ham
sale today! to stop texts call 98912460324 spam
i dont want to go. can we try it a different day? available sat ham
our new mobile video service is live. just install on your phone to start watching. spam
you have won £1000 cash! call to claim your prize. spam
i'll bring it tomorrow. don't forget the milk. ham
wow, is your arm alright. that happened to me one time too ham


In [20]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!
